In [28]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input

In [29]:
# Creating a sample dataset
np.random.seed(42)
start_date = datetime(2023, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(365)]  # One year of data
products = np.random.choice(['Mountain', 'Road', 'Hybrid'], size=365)
quantities = np.random.randint(5, 30, size=365)  # Daily sales
prices = np.random.normal(1, 0.1, size=365) * np.where(products == 'Mountain', 500, np.where(products == 'Road', 800, 650))

df = pd.DataFrame({
    'date_ordered': dates,
    'product': products,
    'quantity': quantities,
    'price': prices
})

# Aggregate data to weekly sales
df['week'] = df['date_ordered'].dt.to_period('W')
weekly_sales = df.groupby(['week', 'product']).agg({'quantity': 'sum', 'price': 'mean'}).reset_index()
weekly_sales['date_ordered'] = weekly_sales['week'].dt.start_time
weekly_sales = weekly_sales.drop('week', axis=1)
print(df)

    date_ordered   product  quantity        price                   week
0     2023-01-01    Hybrid        12   551.552635  2022-12-26/2023-01-01
1     2023-01-02  Mountain        18   533.982983  2023-01-02/2023-01-08
2     2023-01-03    Hybrid        28   642.018731  2023-01-02/2023-01-08
3     2023-01-04    Hybrid        22   692.178061  2023-01-02/2023-01-08
4     2023-01-05  Mountain        19   539.013758  2023-01-02/2023-01-08
..           ...       ...       ...          ...                    ...
360   2023-12-27      Road        21   856.929812  2023-12-25/2023-12-31
361   2023-12-28    Hybrid        13   563.371609  2023-12-25/2023-12-31
362   2023-12-29      Road         5  1020.860955  2023-12-25/2023-12-31
363   2023-12-30      Road        25   782.745158  2023-12-25/2023-12-31
364   2023-12-31    Hybrid        24   628.987756  2023-12-25/2023-12-31

[365 rows x 5 columns]


In [30]:

try:
    # For newer versions of scikit-learn
    column_transformer = ColumnTransformer([
        ('ohe', OneHotEncoder(sparse_output=False), ['product']),
        ('scaler', StandardScaler(), ['quantity', 'price'])
    ])
except TypeError:
    # For older versions of scikit-learn
    column_transformer = ColumnTransformer([
        ('ohe', OneHotEncoder(sparse=False), ['product']),
        ('scaler', StandardScaler(), ['quantity', 'price'])
    ])

# Fit and transform the data
data = column_transformer.fit_transform(weekly_sales)

n_features = data.shape[1]

# Create sequences (use last 4 weeks to predict the next week)
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        end_ix = i + n_steps
        seq_x = data[i:end_ix]
        seq_y = data[end_ix, 1:4]  # Predict quantities for all 3 products
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

n_steps = 4
X, y = create_sequences(data, n_steps)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model definition
model = Sequential([
    Input(shape=(n_steps, n_features)),
    LSTM(64, activation='relu', return_sequences=True),
    LSTM(32, activation='relu'),
    Dense(3)  # Output layer with 3 units (one for each product)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Fit the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

# Model evaluation
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

# Make predictions
predictions = model.predict(X_test)


Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 384ms/step - loss: 0.5722 - mae: 0.4961 - val_loss: 0.4082 - val_mae: 0.4688
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.5501 - mae: 0.5005 - val_loss: 0.3946 - val_mae: 0.4741
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.5426 - mae: 0.5092 - val_loss: 0.3822 - val_mae: 0.4793
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4957 - mae: 0.5068 - val_loss: 0.3702 - val_mae: 0.4854
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5486 - mae: 0.5299 - val_loss: 0.3592 - val_mae: 0.4922
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.5121 - mae: 0.5368 - val_loss: 0.3485 - val_mae: 0.5007
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.5148 - mae: 0.5504 - val_loss: 0.3395 - val_mae: 0.5107
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.4558 - mae: 0.5330 - val_loss: 0.3342 - val_mae: 0.5218
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.5566 - mae: 0.5935 -

In [31]:
def inverse_transform_predictions(column_transformer, X_test, predictions):
    # Get the OneHotEncoder and StandardScaler from the ColumnTransformer
    ohe = column_transformer.named_transformers_['ohe']
    scaler = column_transformer.named_transformers_['scaler']
    
    print(f"Shape of X_test: {X_test.shape}")
    print(f"Shape of predictions: {predictions.shape}")
    
    # Inverse transform the one-hot encoded product
    product_inverse = ohe.inverse_transform(X_test[:, -1, :3])
    print(f"Shape of product_inverse: {product_inverse.shape}")
    
    # Inverse transform the scaled quantities
    try:
        quantities_inverse = scaler.inverse_transform(predictions)
        print(f"Shape of quantities_inverse: {quantities_inverse.shape}")
    except ValueError as e:
        print(f"Error in inverse_transform: {e}")
        print(f"Shape of scaler.scale_: {scaler.scale_.shape}")
        print(f"Shape of scaler.mean_: {scaler.mean_.shape}")
    
    # For debugging, return the shapes
    return {
        'X_test_shape': X_test.shape,
        'predictions_shape': predictions.shape,
        'product_inverse_shape': product_inverse.shape,
    }

# Use the updated function
debug_info = inverse_transform_predictions(column_transformer, X_test, predictions)
print("Debug info:", debug_info)

Shape of X_test: (29, 4, 5)
Shape of predictions: (29, 3)
Shape of product_inverse: (29, 1)
Error in inverse_transform: operands could not be broadcast together with shapes (29,3) (2,) (29,3) 
Shape of scaler.scale_: (2,)
Shape of scaler.mean_: (2,)
Debug info: {'X_test_shape': (29, 4, 5), 'predictions_shape': (29, 3), 'product_inverse_shape': (29, 1)}


In [32]:
def inverse_transform_predictions(column_transformer, X_test, predictions, dates):
    # Get the OneHotEncoder and StandardScaler from the ColumnTransformer
    ohe = column_transformer.named_transformers_['ohe']
    scaler = column_transformer.named_transformers_['scaler']
    
    # Inverse transform the one-hot encoded product
    product_inverse = ohe.inverse_transform(X_test[:, -1, :3])
    
    # Inverse transform the scaled quantities
    quantities_inverse = predictions * scaler.scale_[0] + scaler.mean_[0]
    
    # Combine the results
    result = []
    for date, prods, quants in zip(dates, product_inverse, quantities_inverse):
        result.extend((date, prod, quant) for prod, quant in zip(prods, quants))
    
    return np.array(result, dtype=object)

# Assuming you have a dates array corresponding to your predictions
# If not, you'll need to create this based on your data
dates = pd.date_range(start='2023-01-01', periods=len(predictions), freq='W')

# Use the updated function
predictions_original_scale = inverse_transform_predictions(column_transformer, X_test, predictions, dates)

print("Sample predictions (date, product, quantity):")
for date, product, quantity in predictions_original_scale[:15]:
    print(f"{date.strftime('%Y-%m-%d')}: {product}: {float(quantity):.0f}")

# Additional debugging information
print("\nShape of predictions_original_scale:", predictions_original_scale.shape)

Sample predictions (date, product, quantity):
2023-01-01: Hybrid: 58
2023-01-08: Hybrid: 51
2023-01-15: Mountain: 39
2023-01-22: Road: 55
2023-01-29: Hybrid: 53
2023-02-05: Hybrid: 60
2023-02-12: Road: 46
2023-02-19: Hybrid: 63
2023-02-26: Hybrid: 54
2023-03-05: Mountain: 49
2023-03-12: Hybrid: 53
2023-03-19: Road: 52
2023-03-26: Hybrid: 63
2023-04-02: Road: 44
2023-04-09: Hybrid: 55

Shape of predictions_original_scale: (29, 3)
